# Peptides

In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = 100
pd.options.display.min_rows = 30

In [3]:
DATA_PEPTIDE_INTENSITIES = 'data/peptide_intensities.pkl'

peptides = pd.read_pickle(DATA_PEPTIDE_INTENSITIES)
peptides

Sequence,AAAAAAAAAAAAAAAGAGAGAK,AAAAAAAAAAAAAAATAASAAASAILGGR,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER,AAAAAAAAAAGAAGGR,AAAAAAAAAAGAAGGRGSGPGR,AAAAAAAAAAGAAGGRGSGPGRR,AAAAAAAAAK,AAAAAAAAAPAAAATAPTTAATTAATAAQ,AAAAAAAAAVSR,AAAAAAAAGAFAGR,AAAAAAAGDSDSWDADAFSVEDPVR,AAAAAAAGDSDSWDADAFSVEDPVRK,AAAAAAAK,AAAAAAALESWQAAAPR,AAAAAAALQAK,AAAAAAAPSGGGGGGEEERLEEK,AAAAAAGAASGLPGPVAQGLK,AAAAAAGAGPEMVR,AAAAAAGAGPEMVRGQVFDVGPR,AAAAAAGEAR,AAAAAAGEARR,AAAAAAGPSPGSGPGDSPEGPEGEAPERR,AAAAAAGSGTPR,AAAAAAGSGTPREEEGPAGEAAASQPQAPTSVPGAR,AAAAAASAGSSASSGNQPPQELGLGELLEEFSR,AAAAAATAAAAASIR,AAAAAATAAAAASIRER,AAAAAATAAAAASIRERQTVALK,AAAAAATVLLR,AAAAAAVGPGAGGAGSAVPGGAGPCATVSVFPGAR,AAAAADLANR,AAAAADLANRSGESNTHQDIEEK,AAAAAEQQQFYLLLGNLLSPDNVVR,AAAAAEQQQFYLLLGNLLSPDNVVRK,AAAAAETPEVLRECGCK,AAAAAGAGSGPWAAQEK,AAAAAGLGGGGAGPGPEAGDFLAR,AAAAAGTATSQR,AAAAAGTATSQRFFQSFSDALIDEDPQAALEELTK,AAAAALSQQQSLQER,AAAAAMAEQESAR,AAAAAQGGGGGEPR,AAAAAQGGGGGEPRRTEGVGPGVPGEVEMVK,AAAAAQKQAK,AAAAASAAGPGGLVAGK,AAAAASAAGPGGLVAGKEEK,AAAAASAPQQLSDEELFSQLR,AAAAASAPQQLSDEELFSQLRR,AAAAASASQDELNQLER,AAAAASHLNLDALR,...,YYSFFDLNPK,YYSHVDYR,YYSIASSSK,YYSLDELSEK,YYSLWIPEQAR,YYSQPILVK,YYSSENTRPK,YYSSEYHYVGGFYGGCNEALMK,YYSSHYMTLVVQSK,YYSVNVPIQDGIQDEK,YYSYGLEK,YYTALYR,YYTEFPTVLDITAEDPSK,YYTGNYDQYVK,YYTGVVNNNEMVALQR,YYTGVVNNNEMVALQRDPNNPYDK,YYTLEEIQK,YYTLFGR,YYTLNGSK,YYTMHLTTLCNTSLDNPTQR,YYTPTISR,YYTPTISRER,YYTPVLAK,YYTPVPCESATAK,YYTRLGNDFHTNK,YYTSASGDEMVSLK,YYTSESK,YYTVFDR,YYTVFDRDNNR,YYTVFDRDNNRVGFAEAAR,YYTYMIMNK,YYVDVPQEEVPFR,YYVLNALK,YYVNSLQHR,YYVQLEQLK,YYVSGVDNK,YYVSPSDLLDDK,YYVTIIDAPGHR,YYVTIIDAPGHRDFIK,YYWGGQYTWDMAK,YYYAVVDCDSPETASK,YYYAVYDMVVR,YYYDGDMICK,YYYDGKDYIEFNKEIPAWVPFDPAAQITK,YYYECVLFVVR,YYYGNLIAVSNSFLAYAIR,YYYIPQYK,YYYQGCASWK,YYYVPADFVEYEK,YYYVQNVYTPVDEHVYPDHR
20180713_QE8_nLC5_ASD_QC_Hela1_peptides.txt,NaN,NaN,NaN,0.0,103770000.0,NaN,NaN,1.113700e+09,NaN,NaN,NaN,161350000.0,NaN,NaN,2.427700e+09,NaN,315150000.0,47772000.0,139290000.0,NaN,54774000.0,NaN,NaN,NaN,NaN,NaN,51638000.0,NaN,NaN,195040000.0,NaN,NaN,NaN,9.982000e+08,NaN,NaN,NaN,145420000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,171010000.0,NaN,70882000.0,NaN,NaN,NaN,NaN,69369000.0,NaN,NaN,NaN,NaN,92948000.0,59060000.0,NaN,NaN,0.0,147250000.0,NaN,NaN,104320000.0,39883000.0,NaN,32042000.0,190470000.0,1.207400e+09,NaN,NaN,NaN,109160000.0,NaN,NaN,NaN,NaN,NaN,NaN,21919000.0,8.215900e+08,1.428700e+10,NaN,NaN,NaN,NaN,NaN,NaN,5482200.0,NaN,NaN,NaN,NaN
20180713_QE8_nLC5_ASD_QC_Hela2_20190226172112_peptides.txt,18280000.0,NaN,NaN,23154000.0,92718000.0,NaN,NaN,1.021500e+09,NaN,NaN,NaN,133950000.0,NaN,NaN,3.002200e+09,NaN,261180000.0,NaN,101570000.0,69406000.0,56584000.0,NaN,NaN,NaN,NaN,NaN,56967000.0,NaN,NaN,132140000.0,NaN,NaN,34317000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14309000.0,NaN,NaN,NaN,NaN,NaN,NaN,109680000.0,NaN,NaN,NaN,NaN,103860000.0,121340000.0,NaN,NaN,110910000.0,242410000.0,NaN,NaN,124930000.0,NaN,NaN,NaN,575470000.0,1.408100e+09,NaN,NaN,NaN,164260000.0,NaN,NaN,166870000.0,NaN,NaN,NaN,NaN,1.123500e+09,1.874800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75013000.0,26265000.0,NaN,NaN
20180713_QE8_nLC5_ASD_QC_Hela2_peptides.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190103_QE8_nLC0_LiNi_QC_MNT_15cm_Hela_01_20190104110509_peptides.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.221000e+09,NaN,NaN,NaN,176000000.0,NaN,NaN,2.442400e+09,NaN,231780000.0,45597000.0,NaN,NaN,100090000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29533000.0,NaN,NaN,NaN,NaN,NaN,97363000.0,NaN,32325000.0,80621000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114170000.0,NaN,112690000.0,108990000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [4]:
N_MIN_OBS = 10
mask_min_obsevation = peptides.notna().sum() >= N_MIN_OBS
mask_min_obsevation.sum()

122552

## Look at sequences

In [5]:
class SequenceAnalyser():
    
    def __init__(self, sequences : pd.Series):
        if not isinstance(sequences, pd.Series):
            raise ValueError("Please provide a pandas.Series, not {}".format(type(sequences)))
        self.sequences = sequences
    
    def calc_counts(self,n_characters):
        return self.sequences.str[:n_characters].value_counts()
    
    def length(self):
        return self.sequences.str.len().sort_values()
    

In [6]:
sequences = SequenceAnalyser(peptides.columns.to_series())
sequences.length()

Sequence
VEECTLK                                                     7
VQVLEPR                                                     7
VWLQSEK                                                     7
TYEDLKR                                                     7
YDEPLGK                                                     7
TMIEDRK                                                     7
GLQVYIR                                                     7
VELSQLR                                                     7
DLGEIGR                                                     7
EMAGDNK                                                     7
LQQEWLK                                                     7
SGETSVR                                                     7
TYEDIDK                                                     7
VQVFLPR                                                     7
QVTGVTR                                                     7
                                                           ..

In [7]:
import ipywidgets as w
w.interact(sequences.calc_counts, n_characters=w.IntSlider(value=4, min=1, max=55))

interactive(children=(IntSlider(value=4, description='n_characters', max=55, min=1), Output()), _dom_classes=(…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [8]:
sequences_p4 = sequences.calc_counts(4)
display(sequences_p4.head())
sequences_p4.loc[sequences_p4.isin(('CON_','REV_'))].sort_index()

AAAA    117
ELEE     60
LLEE     59
LLEA     55
KLEE     54
Name: Sequence, dtype: int64

Series([], Name: Sequence, dtype: int64)

What to do when 


```
AAAAAAAAAAGAAGGRGSGPGR
AAAAAAAAAAGAAGGRGSGPGRR

AAAANSGSSLPLFDCPTWAGKPPPGLHLDVVK
AAAANSGSSLPLFDCPTWAGKPPPGLHLDVVKGDK
```



## NextUp
- check with some reference list of peptides